In [2]:
import cv2
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from skimage import feature
from sklearn.metrics import (f1_score, precision_score, recall_score, 
accuracy_score, confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier

In [3]:
def gammaCorrection(i):
  return np.array(255*(i / 255) ** 1, dtype = 'uint8') 

#############################################################

def pixelVal(pix, ilo, ihi, dlo, dhi): 
  return ((dhi - dlo)/(ihi - ilo)) * (pix - ilo) + dlo 

pixelVal_vec = np.vectorize(pixelVal) 

def contrast(img):
  dhi = 200
  dlo = 0
  ihi = np.max(img)
  ilo = np.min(img)    
  return pixelVal_vec(img, ilo, ihi, dlo, dhi) 

In [17]:
def criarListaImagens(listaCategorias):
  lista = []
  for categoria in listaCategorias:
    path = os.path.join(data_dir, categoria)
    class_num = listaCategorias.index(categoria)
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path, img))
        new_array = cv2.resize(img_array, (img_size, img_size))
        lista.append([new_array, class_num])
      except Exception as e:
        print(e)
        pass
  return lista

#----------------------------------------------------------------------#

def criaListaTreinoTeste(lista):
  classes = []
  imagens = []
  for imagem in lista:
    imagens.append(imagem[0])
    classes.append(imagem[1])
  return imagens, classes

#----------------------------------------------------------------------#

def obtemCanaisCoresRGB(listaOrigem):
    listaDestino = []
    for imagem in listaOrigem:
        img_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)        
        r, g, b = cv2.split(img_rgb)
        aux = []
        aux.append(contrast(b))
        aux.append(contrast(g))
        aux.append(contrast(r))
        aux.append(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY))
        listaDestino.append(aux)
    return listaDestino

#----------------------------------------------------------------------#

def obterCanaisCores(listaOrigem, cvt):
  listaDestino = []
  for imagem in listaOrigem:    
    i = cv2.cvtColor(imagem, cvt)
    h, s, v = cv2.split(i)
    aux = []
    aux.append(contrast(h))
    aux.append(contrast(s))
    aux.append(contrast(v))
    aux.append(cv2.cvtColor(i, cv2.COLOR_BGR2GRAY))
    listaDestino.append(aux)

  return listaDestino

#----------------------------------------------------------------------#

class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

#----------------------------------------------------------------------#

def obtemHistogramaCanais(listaImagens, lbp):
  canal_1 = []
  canal_2 = []
  canal_3 = []
  canal_cinza = []
  for imagem in listaImagens:
    canal_1.append(lbp.describe(imagem[0]))
    canal_2.append(lbp.describe(imagem[1]))
    canal_3.append(lbp.describe(imagem[2]))
    canal_cinza.append(lbp.describe(imagem[3]))
  return canal_1, canal_2, canal_3, canal_cinza

#----------------------------------------------------------------------#

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
        max_iter=-1, probability=False, random_state=109, shrinking=True, tol=0.001,
        verbose=False),
    SVC(C=10000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovo', degree=3, gamma=1, kernel='rbf', max_iter=-1,
        probability=False, random_state=109, shrinking=True, tol=0.001,verbose=False),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(class_weight=None, criterion='entropy',
        max_depth=3, max_features=None, max_leaf_nodes=None,
        min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=50, min_samples_split=2,
        min_weight_fraction_leaf=0.0,
        random_state=None, splitter='best'),
    RandomForestClassifier(bootstrap=False, class_weight=None,
        criterion='gini', max_depth=30, max_features='auto',
        max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None,
        min_samples_leaf=4, min_samples_split=2,
        min_weight_fraction_leaf=0.0, n_estimators=600,
        n_jobs=None, oob_score=False, random_state=None),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]

#----------------------------------------------------------------------#

def xgboost(listaTreino, listaTeste, classesTreino, classesTeste):
    m = XGBClassifier(colsample_bytree= 0.7, 
        gamma= 0.2, learning_rate= 0.15, max_depth= 5, 
        min_child_weight= 1, eta= 0.3 ).fit(listaTreino, classesTreino)
    preds = m.predict(listaTeste)
    y_pred = m.predict(listaTeste)     
    acc = accuracy_score(classesTeste, y_pred)
    f1  = f1_score(classesTeste, y_pred, average='weighted')
    cm  = confusion_matrix(classesTeste, y_pred)
    ps  = precision_score(classesTeste, y_pred, average='weighted')
    rs  = recall_score(classesTeste, y_pred, average='weighted')
    return acc, f1, cm, ps, rs

#----------------------------------------------------------------------#

def utilizarClassificadores(listaTreino, classesTreino, listaTeste, classesTeste, namesClassifiers, classifiers):
    listaAccuracy = {}
    listaCM = {}
    listaF1Score = {}
    listaPS = {}
    listaRS = {}
    for i, c in enumerate(classifiers):
        m = c.fit(listaTreino, classesTreino)
        y_pred = m.predict(listaTeste)         
        listaAccuracy[namesClassifiers[i]] = accuracy_score(classesTeste, y_pred)
        listaF1Score[namesClassifiers[i]]  = f1_score(classesTeste, y_pred, average='weighted')
        listaCM[namesClassifiers[i]]       = confusion_matrix(classesTeste, y_pred)
        listaPS[namesClassifiers[i]]       = precision_score(classesTeste, y_pred, average='weighted')
        listaRS[namesClassifiers[i]]       = recall_score(classesTeste, y_pred, average='weighted')
    listaAccuracy["XGBoost"], listaF1Score["XGBoost"], listaCM["XGBoost"], listaPS["XGBoost"], listaRS["XGBoost"] = xgboost(np.asarray(listaTreino), np.asarray(listaTeste), np.asarray(classesTreino), np.asarray(classesTeste))
    return listaAccuracy , listaF1Score, listaCM, listaPS, listaRS



In [15]:
data_dir = r"C:\Users\Rosana\Documents\Mestrado\DataSets\RIM_ONE_v2"
categorias = ["glaucoma","normal"]
img_size = 200
listaImagens = criarListaImagens(categorias)
random.shuffle(listaImagens)

In [16]:
imagens, classes = criaListaTreinoTeste(listaImagens)

imagens_gamma = []
for img in imagens:
    imagens_gamma.append(gammaCorrection(img))
    
imagens_treino, imagens_teste, classes_treino, classes_teste = train_test_split(imagens_gamma, classes, test_size=0.33, random_state=42)

In [34]:
cv2.imshow('image',imagens[0])

In [18]:
listaRgbTreino = obtemCanaisCoresRGB(imagens_treino)
listaHsvTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2HSV)
listaLabTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2LAB)
listaLuvTreino = obterCanaisCores(imagens_treino, cv2.COLOR_BGR2LUV)

listaRgbTeste = obtemCanaisCoresRGB(imagens_teste)
listaHsvTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2HSV)
listaLabTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2LAB)
listaLuvTeste = obterCanaisCores(imagens_teste, cv2.COLOR_BGR2LUV)

In [19]:
def classificarImagens(pontos, raio):
    _lbp = LocalBinaryPatterns(pontos, raio)

    lista_treino_rgb_b, lista_treino_rgb_g, lista_treino_rgb_r, lista_treino_rgb_gray = obtemHistogramaCanais(listaRgbTreino, _lbp)
    lista_treino_hsv_h, lista_treino_hsv_s, lista_treino_hsv_v, _ = obtemHistogramaCanais(listaHsvTreino, _lbp)
    lista_treino_lab_l, lista_treino_lab_a, lista_treino_lab_b, _ = obtemHistogramaCanais(listaLabTreino, _lbp)
    lista_treino_luv_l, lista_treino_luv_u, lista_treino_luv_v, _ = obtemHistogramaCanais(listaLuvTreino, _lbp)

    lista_teste_rgb_b, lista_teste_rgb_g, lista_teste_rgb_r, lista_teste_rgb_gray = obtemHistogramaCanais(listaRgbTeste, _lbp)
    lista_teste_hsv_h, lista_teste_hsv_s, lista_teste_hsv_v, _ = obtemHistogramaCanais(listaHsvTeste, _lbp)
    lista_teste_lab_l, lista_teste_lab_a, lista_teste_lab_b, _ = obtemHistogramaCanais(listaLabTeste, _lbp)
    lista_teste_luv_l, lista_teste_luv_u, lista_teste_luv_v, _ = obtemHistogramaCanais(listaLuvTeste, _lbp)

    linha = []
    listaF1Score = []
    listaCM = [] 
    conf_matrix = []
    listaPS = []
    listaRS = []

    #('------------------ RGB ----------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_gray, classes_treino, lista_teste_rgb_gray, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - R ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_r, classes_treino, lista_teste_rgb_r, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - G ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_g, classes_treino, lista_teste_rgb_g, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ RGB - B ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_rgb_b, classes_treino, lista_teste_rgb_b, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)

    #('------------------ HSV - H ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_h, classes_treino, lista_teste_hsv_h, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ HSV - S ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_s, classes_treino, lista_teste_hsv_s, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    #('------------------ HSV - V ------------')
    l, f1, cm, ps, rs = pd.Series(utilizarClassificadores(lista_treino_hsv_v, classes_treino, lista_teste_hsv_v, classes_teste, names, classifiers))
    linha.append(l)
    listaF1Score.append(f1)
    listaCM.append(cm)
    listaPS.append(ps)
    listaRS.append(rs)
    
    return linha, listaF1Score, listaCM, listaPS, listaRS

In [20]:
def obterResultados (pontos, raio):
    linhas, listaF1Score, listaCM, listaPS, listaRS = classificarImagens(pontos, raio)
    print('------ %s pontos - %s raio ------' % (pontos, raio))
    print('--- Acuracia ---')
    print(pd.DataFrame(linhas))
    print('--- F1 ---')
    print(pd.DataFrame(listaF1Score))
    print('--- Precision Score ---')
    print(pd.DataFrame(listaPS))
    print('--- Recall Score ---')
    print(pd.DataFrame(listaRS))
    print('--- Matriz confusao ---')
    print(pd.DataFrame(listaCM))
    print('---------------------------------')
    

In [21]:
obterResultados(4,1)
obterResultados(8,1)
obterResultados(8,2)
obterResultados(12,2)
obterResultados(16,2)
obterResultados(16,3)
obterResultados(24,3)
obterResultados(24,8)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sk

------ 4 pontos - 1 raio ------
--- Acuracia ---
   Nearest Neighbors  Linear SVM   RBF SVM  Gaussian Process  Decision Tree  \
0           0.748344    0.721854  0.768212          0.754967       0.721854   
1           0.794702    0.715232  0.754967          0.781457       0.682119   
2           0.735099    0.728477  0.741722          0.774834       0.701987   
3           0.748344    0.741722  0.774834          0.774834       0.728477   
4           0.629139    0.642384  0.655629          0.642384       0.635762   
5           0.735099    0.735099  0.781457          0.754967       0.735099   
6           0.794702    0.715232  0.754967          0.781457       0.682119   

   Random Forest  Neural Net  AdaBoost  Naive Bayes       QDA   XGBoost  
0       0.728477    0.609272  0.708609     0.695364  0.682119  0.688742  
1       0.774834    0.602649  0.701987     0.728477  0.728477  0.748344  
2       0.761589    0.615894  0.748344     0.695364  0.735099  0.754967  
3       0.781457    0.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'prec

------ 8 pontos - 1 raio ------
--- Acuracia ---
   Nearest Neighbors  Linear SVM   RBF SVM  Gaussian Process  Decision Tree  \
0           0.741722    0.695364  0.761589          0.748344       0.728477   
1           0.728477    0.715232  0.715232          0.735099       0.748344   
2           0.741722    0.721854  0.735099          0.774834       0.715232   
3           0.682119    0.741722  0.774834          0.761589       0.774834   
4           0.635762    0.655629  0.695364          0.629139       0.622517   
5           0.735099    0.735099  0.768212          0.741722       0.781457   
6           0.728477    0.715232  0.715232          0.735099       0.748344   

   Random Forest  Neural Net  AdaBoost  Naive Bayes       QDA   XGBoost  
0       0.735099    0.615894  0.735099     0.688742  0.807947  0.761589  
1       0.708609    0.629139  0.781457     0.721854  0.721854  0.715232  
2       0.735099    0.615894  0.754967     0.682119  0.682119  0.768212  
3       0.688742    0.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined 

...

------ 16 pontos - 3 raio ------
--- Acuracia ---
   Nearest Neighbors  Linear SVM   RBF SVM  Gaussian Process  Decision Tree  \
0           0.774834    0.774834  0.761589          0.781457       0.728477   
1           0.701987    0.735099  0.801325          0.788079       0.602649   
2           0.748344    0.807947  0.768212          0.794702       0.741722   
3           0.741722    0.774834  0.794702          0.781457       0.721854   
4           0.569536    0.774834  0.774834          0.761589       0.735099   
5           0.728477    0.708609  0.715232          0.397351       0.675497   
6           0.701987    0.735099  0.801325          0.788079       0.602649   

   Random Forest  Neural Net  AdaBoost  Naive Bayes       QDA   XGBoost  
0       0.774834    0.602649  0.761589     0.774834  0.761589  0.741722  
1       0.708609    0.609272  0.721854     0.688742  0.774834  0.682119  
2       0.814570    0.582781  0.735099     0.774834  0.748344  0.768212  
3       0.788079    0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'pr

------ 24 pontos - 3 raio ------
--- Acuracia ---
   Nearest Neighbors  Linear SVM   RBF SVM  Gaussian Process  Decision Tree  \
0           0.768212    0.761589  0.801325          0.788079       0.715232   
1           0.708609    0.748344  0.761589          0.721854       0.649007   
2           0.754967    0.754967  0.788079          0.788079       0.754967   
3           0.754967    0.801325  0.821192          0.774834       0.741722   
4           0.609272    0.735099  0.735099          0.754967       0.622517   
5           0.715232    0.682119  0.741722          0.397351       0.668874   
6           0.708609    0.748344  0.761589          0.721854       0.649007   

   Random Forest  Neural Net  AdaBoost  Naive Bayes       QDA   XGBoost  
0       0.768212    0.602649  0.721854     0.781457  0.781457  0.781457  
1       0.701987    0.675497  0.728477     0.701987  0.741722  0.708609  
2       0.781457    0.602649  0.774834     0.781457  0.741722  0.761589  
3       0.761589    0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:693: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined 

------ 24 pontos - 8 raio ------
--- Acuracia ---
   Nearest Neighbors  Linear SVM   RBF SVM  Gaussian Process  Decision Tree  \
0           0.701987    0.781457  0.794702          0.788079       0.688742   
1           0.688742    0.814570  0.860927          0.847682       0.754967   
2           0.695364    0.774834  0.768212          0.741722       0.662252   
3           0.701987    0.781457  0.801325          0.794702       0.688742   
4           0.596026    0.695364  0.754967          0.662252       0.635762   
5           0.655629    0.715232  0.754967          0.754967       0.668874   
6           0.688742    0.814570  0.860927          0.847682       0.754967   

   Random Forest  Neural Net  AdaBoost  Naive Bayes       QDA   XGBoost  
0       0.721854    0.602649  0.715232     0.701987  0.708609  0.741722  
1       0.801325    0.602649  0.768212     0.748344  0.821192  0.801325  
2       0.728477    0.609272  0.735099     0.715232  0.728477  0.781457  
3       0.834437    0